# Calibración de $\gamma$ y $\sigma$ en el Modelo de Hull-White

Vamos a calibrar $\gamma$ y $\gamma$ utilizando precios de Caps y las fórmulas para Calls y Puts sobre un bono cupón cero que se obtienen en este modelo más la equivalencia entre una Put sobre un bono cupón cero y un caplet.

Recordemos la fórmula para una Put que vence en $T_{O}$, strike igual a $K$ sobre el bono cupón cero $Z(r_{t}, t, T_{B})$ .

$$Put\left(r_{0},0\right)=K Z(r_{0},0, T_{O})N(-d_{2})-Z(r_{0},0, T_{B})N(-d_{1})$$

$$d_{1}=\frac{1}{S_{Z}(T_{O},T_{B})}ln\left(\frac{Z(r_{0},0, T_{B})}{Z(r_{0},0, T_{O})}\right)+\frac{S_{Z}(T_{O})}{2}$$

$$d_{2}=d_{1}-S_{Z}(T_{O},T_{B})$$

$$S_{Z}(T_{O},T_{B})=B(T_{O},T_{B})\sqrt{\frac{\sigma^2}{2\gamma}\left(1-e^{2\gamma T_{O}}\right)}$$

De esta fórmula se puede deducir la fórmula para un caplet de strike $r_{K}$ sobre una tasa forward entre $T$ y $T-\Delta$.

$$V\left(r_{0},0\right)=M\left[KZ(r_{0},0, T-\Delta)N(-d_{2})-Z(r_{0},0, T)N(-d_{1})\right]$$

$$M = N\left(1+r_{K}\Delta\right)$$

$$K=\frac{1}{1+r_{K}\Delta}$$

$$d_{1}=\frac{1}{S_{Z}(T-\Delta,T)}ln\left(\frac{Z(r_{0},0, T)}{Z(r_{0},0, T-\Delta)}\right)+\frac{S_{Z}(T-\Delta,T)}{2}$$

$$d_{2}=d_{1}-S_{Z}(T-\Delta,T)$$

Subamos data para hacer el ejercicio, esta se encuentra en un archivo Excel que cargaremos a un DataFrame de **pandas**.

In [5]:
import pandas as pd
data = pd.read_excel("./Data.xlsx")
data

,Maturity,SwapRates,Df,CapPrices
0,0.25,0.021800,0.994580,0.000000
1,0.50,0.023177,0.988509,0.000456
2,0.75,0.024420,0.981899,0.001059
3,1.00,0.025550,0.974834,0.001859
4,1.25,0.026586,0.967384,0.002887
5,1.50,0.027546,0.959598,0.004157
6,1.75,0.028451,0.951503,0.005662
7,2.00,0.029320,0.943109,0.007364
8,2.25,0.030167,0.934418,0.009201
9,2.50,0.030991,0.925457,0.011129


Tenemos:

- Maturity: plazos en fracción de año
- SwapRates: tasas swap a esos plazos (la periodicidad de los swaps coincide con la periodicidad de los plazos),
- Df: factores de descuento obtenidos haciendo el bootstrapping de las tasas swap
- CapPrices: los precios de Caps a los distintos plazos.
 - El Cap tiene strike igual a la tasa swap.
 - El Cap a t = 0.25 no está definido (de ahí el 0),
 - el Cap a t = 0.50 tiene un caplet entre t = 0.25 y t = 0.50,
 - el Cap a t = 0.75 tiene un caplet entre t = 0.25 y t = 0.50 y un caplet entre t = 0.50 y t = 0.75, y así sucesivamente.

Implementemos la función para el valor del Caplet:
- Primero se implementa $B$
- Luego se implementa $S_{Z}$
- Finalmente se implementa la fórmula del caplet

In [6]:
import math
from scipy.stats import norm

# Coeficiente B del modelo de HW
def b_hw(gamma, t, T):
    aux = 1 - math.exp(- gamma * (T - t))
    return aux / gamma

# Volatilidad del bono cupón cero en HW
def sz(gamma, sigma, t, T):
    b = b_hw(gamma, t, T)
    return b * math.sqrt((sigma**2)/(2 * gamma)*(1 - math.exp(-2 * gamma* t)))

# Valor del caplet
def hw_caplet(r0, rK, To, Tb, zo, zb, gamma, sigma):
    delta = Tb - To
    M = 1 + rK * delta # Suponemos N = 1
    K = 1 / M
    sZ = sz(gamma, sigma, To, Tb)
    d1 = 1 / sZ * math.log(zb/(K * zo)) + .5 * sZ
    d2 = d1 - sZ
    return M * (K * zo * norm.cdf(-d2) - zb * norm.cdf(-d1))    

In [7]:
# Calculemos un ejemplo
zo = data.Df.values[data.Maturity.values == .25][0]
print("zo: " + str(zo))

r0 = -math.log(zo) / .25
print("r0: " + str(r0))

zb = data.Df.values[data.Maturity.values == .5][0]
print("zb: " + str(zb))

rK = data.SwapRates.values[data.Maturity.values == .5][0]
print("rK: " + str(rK))

gamma = 4
print("gamma: " + str(gamma))

sigma = .05
print("sigma: " + str(sigma))

print("caplet: " + str(hw_caplet(r0, rK, .25, .5, zo, zb, gamma, sigma)))

zo: 0.994579541499
r0: 0.02174080995975708
zb: 0.9885094864
rK: 0.023177
gamma: 4
sigma: 0.05
caplet: 0.00121076776372


Veamos ahora como calcular el valor de un Cap conociendo el plazo al vencimiento del Cap, el DataFrame data y los parámetros $\gamma$ y $\sigma$.

In [8]:
# Valor Cap
def valor_cap(r0, data, maturity, gamma, sigma):
    if maturity < .5:
        return "Plazo mínimo es 0.5"
    data_ok = data.values[data.Maturity <= maturity]
    # print(data_ok)
    num_caplets = len(data_ok) - 1
    rK = data_ok[num_caplets][1]
    mkt_value = data_ok[num_caplets][3]
    data_ok = data_ok[::-1]
    # print(data_ok)
    cap = 0.0
    for i in range(0, num_caplets):
        cap += hw_caplet(r0, rK, data_ok[i + 1][0], data_ok[i][0], data_ok[i + 1][2], data_ok[i][2],
                        gamma, sigma)
    return cap, mkt_value

In [9]:
# Probamos
print(valor_cap(r0, data, .5, gamma, sigma))

(0.001210767763720441, 0.00045600000000000003)


In [10]:
# Definimos ahora una función con el error cuadrático total
def error_caps(gamma_sigma, r0, data):
    error = 0.0
    inicio = .5
    incremento = .25
    for i in range(0, 19):
        maturity = inicio + i * incremento
        v = valor_cap(r0, data, maturity, gamma_sigma[0], gamma_sigma[1])
        error += 1000 * (v[0] - v[1])**2
    return error

In [11]:
# Calcular error
import numpy as np
gamma_sigma = np.array([gamma, sigma])
error_caps(gamma_sigma, r0, data)

0.27008503301798237

In [12]:
# Optimizar
from scipy.optimize import fmin_slsqp as min
min(error_caps, gamma_sigma, args=(r0, data), bounds=[(0.01,100),(0.00325, 100)], acc=1e-12)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.010851564789411058
            Iterations: 13
            Function evaluations: 56
            Gradient evaluations: 13


array([ 0.01      ,  0.01083915])

In [17]:
# Justificación para el lower bound en sigma
vol_diaria = .0002 # Vol diaria de 2 pip
vol_anual = vol_diaria * math.sqrt(264)
print("vol_anual (en pips): " + str(vol_anual * 10000))

vol_anual (en pips): 32.49615361854384
